In [70]:
from selenium import webdriver
import selenium
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import datetime
from datetime import datetime
import IntellidealerCredentials
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import glob
import shutil
import pandas as pd

today = datetime.now().strftime("%Y-%m-%d %I-%M-%S %p")

# Using Chrome to access web
driver = webdriver.Chrome()
# Open the website
driver.get('http://pfwf6049.cdkglobal-id.net:20209/intellidealer/index.html?_cb=20221220090042#/login?library=PFWF6049&redirectedPort=20209')
driver.set_window_size(1024, 600)
driver.maximize_window()

import time 
time.sleep(5)
# Select the id box
id_box = driver.find_element(By.ID, "cdk-text-1")
id_box.send_keys(IntellidealerCredentials.cdk_id)
pass_box = driver.find_element(By.ID, "cdk-password-2")
pass_box.send_keys(IntellidealerCredentials.cdk_password)
login_button = driver.find_element(By.ID, "login-button")
login_button.click()


time.sleep(10)

# Click Product Support from side panel
product = driver.find_element(By.XPATH, "//span[normalize-space()='Management Central']")
achains = ActionChains(driver)
achains.move_to_element(product).perform()

time.sleep(3)
# Click 'Parts' Option from the submenu
menu_element = driver.find_element(By.XPATH, "//span[normalize-space()='Utilities']")
menu_element.click()

time.sleep(3)
# Click on 'Parts Profile' submenu
sub_menu_element = driver.find_element(By.XPATH, "//span[normalize-space()='Data Miner']")
sub_menu_element.click()

In [71]:
def DownloadTodaysCSV():
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(3)
    # Switching to frame
    element = driver.find_element(By.XPATH, '//*[@id="lansaFrame"]')
    driver.switch_to.frame(element)
    time.sleep(2)
    xpath = '//*[@id="angularWrap"]/form/center/table/tbody/tr[2]/td/table[1]/tbody/tr[2]/td[3]/input'
    clear_user = driver.find_element(By.XPATH, xpath)
    clear_user.clear()
    
    # Enter BRP Subs in Query Field and Hit Go
    xpath = '//*[@id="angularWrap"]/form/center/table/tbody/tr[2]/td/table[1]/tbody/tr[3]/td[3]/input'
    enter_query = driver.find_element(By.XPATH, xpath)
    enter_query.clear()
    enter_query.send_keys('BRPSUBS')

    xpath = '//*[@id="angularWrap"]/form/center/table/tbody/tr[2]/td/table[1]/tbody/tr[4]/td[4]/a'
    hit_go = driver.find_element(By.XPATH, xpath)
    hit_go.click()

    # Running Query Steps will involved hovering over the query name to get Run option and then clicking that
    time.sleep(2)
    driver.switch_to.window(driver.window_handles[0])
    element = driver.find_element(By.XPATH, '//*[@id="lansaFrame"]')
    driver.switch_to.frame(element)
    time.sleep(2)
    xpath = '//*[@id="custheader"]/table/tbody/tr[2]/td[1]/a'
    query_hover = driver.find_element(By.XPATH, xpath)
    achains = ActionChains(driver)
    achains.move_to_element(query_hover).perform()

    time.sleep(3)
    # Click on 'Parts Profile' submenu
    query_run = driver.find_element(By.XPATH, '//*[@id="HM_Item8_3"]')
    query_run.click()
    time.sleep(4)
    # Download CSV
    driver.switch_to.window(driver.window_handles[1])
    xpath = '/html/body/form[1]/center/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td/a'
    download_csv = driver.find_element(By.XPATH, xpath)
    download_csv.click()

    print('Todays CSV Downloaded')
    time.sleep(10)
    # Closing downloaded window after donwloads complete.
    driver.close() 
    driver.switch_to.window(driver.window_handles[0])
    driver.close()

In [72]:
def MoveCSVtoFolder():
    global folder_path
    # Get the current date
    today = datetime.now().strftime("%Y-%m-%d %I-%M-%S %p")

    # Format the date as a string
    date_string = today

    # Create a folder with today's date
    folder_path = f'H:\Intellidealer Webcontrol Projects\Parts Sub Deletion\DM File Downloads\{date_string}'

    os.makedirs(folder_path)
    # The downloaded file will be thrown into this location from where we will read it
    download_dir = "C:/Users/AGERA/Downloads"

    # Searching for the file in the directory based on latest modification time
    list_of_files = glob.glob(download_dir + '/*')
    latest_file = max(list_of_files, key=os.path.getctime)
    print(f'Path of Todays CSV File Downloaded {latest_file}')


    # Transferring the file from downloads folder to Shopify folder
    shutil.move(latest_file, folder_path+'\\PartsToBeDeleted.csv')
    print(f'******** Todays BRPSubs Dataminer file moved to {folder_path}******')

In [73]:
DownloadTodaysCSV()

Todays CSV Downloaded


In [74]:
MoveCSVtoFolder()

Path of Todays CSV File Downloaded C:/Users/AGERA/Downloads\DataminerFile (30).csv
******** Todays BRPSubs Dataminer file moved to H:\Intellidealer Webcontrol Projects\Parts Sub Deletion\DM File Downloads\2023-09-11 09-28-27 AM******


In [75]:
# Importing first downloaded file with headers to be used only for giving column names to future CSVs from Data Miner
df_for_columns = pd.read_csv('H:\Intellidealer Webcontrol Projects\Parts Sub Deletion\PartsSubsToBeDeleted.csv')

# Reading todays Data Miner CSV file

df = pd.read_csv(folder_path+'\\PartsToBeDeleted.csv', names=df_for_columns.columns)
print(df)

       Br  Part Number:          Part  Inventory
0       7  A11350116001  A11350116001          0
1       7  A11359116001  A11350116001          0
2       7  A16950179000     709000053          0
3       7  A16950179000     709000082          0
4       7  A17100179000     707800103          0
...    ..           ...           ...        ...
44659   3        779139       9779139          0
44660   6        779139       9779139          0
44661   7        779139       9779139          0
44662   8        779139       9779139          3
44663   9        779139       9779139          0

[44664 rows x 4 columns]


In [76]:
from selenium import webdriver
import selenium
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import datetime
from datetime import datetime
import IntellidealerCredentials

today = datetime.now().strftime("%Y-%m-%d %I-%M-%S %p")

# Using Chrome to access web
driver = webdriver.Chrome()
# Open the website
driver.get('http://pfwf6049.cdkglobal-id.net:20209/intellidealer/index.html?_cb=20221220090042#/login?library=PFWF6049&redirectedPort=20209')
driver.set_window_size(1024, 600)
driver.maximize_window()

import time 
time.sleep(5)
# Select the id box
id_box = driver.find_element(By.ID, "cdk-text-1")
id_box.send_keys(IntellidealerCredentials.cdk_id)
pass_box = driver.find_element(By.ID, "cdk-password-2")
pass_box.send_keys(IntellidealerCredentials.cdk_password)
login_button = driver.find_element(By.ID, "login-button")
login_button.click()


time.sleep(10)
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Navigating to Parts Profile Window from the Intellidealer Landing Page

# Click Product Support from side panel
product = driver.find_element(By.XPATH, "//span[normalize-space()='Product Support']")
achains = ActionChains(driver)
achains.move_to_element(product).perform()

time.sleep(3)
# Click 'Parts' Option from the submenu
menu_element = driver.find_element(By.XPATH, "//span[normalize-space()='Parts']")
menu_element.click()

time.sleep(3)
# Click on 'Parts Profile' submenu
sub_menu_element = driver.find_element(By.XPATH, "//span[normalize-space()='Parts Profile']")
sub_menu_element.click()

In [77]:
# Creating a function DeleteSubs which contains sequence of steps to Search Part and clear out Subs Info for that Part
# The function will work only when an update profile window is opened for a branch

def DeleteSubs():
    time.sleep(2)
    driver.switch_to.window(driver.window_handles[2])

    
    # Entering Part Number
    xpath = '//*[@id="wrkprt"]'
    enter_part = driver.find_element(By.XPATH, xpath)
    enter_part.clear()
    enter_part.send_keys(sub_df['Part Number:'][i])

    # Hit Go for searching the entered Part
    xpath = '//*[@id="tab_container"]/table[1]/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td[4]/a'
    hit_go = driver.find_element(By.XPATH, xpath)
    hit_go.click()

    # Click on the subs tab
    time.sleep(2)
    xpath = '//*[@id="tabs"]/ul/li[4]/a'
    subs_tab = driver.find_element(By.XPATH, xpath)
    subs_tab.click()

    time.sleep(2)
    # Clear the contents of Substitute Part 
    xpath = '/html/body/form/center/table/tbody/tr[2]/td[2]/table/tbody/tr/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td[1]/input'
    sub_part_field = driver.find_element(By.XPATH, xpath)
    sub_part_field.clear()

    # Clear Quantity Field  
    xpath = '/html/body/form/center/table/tbody/tr[2]/td[2]/table/tbody/tr/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td[2]/input'
    qty_field = driver.find_element(By.XPATH, xpath)
    qty_field.clear()

    # Hit Save
    xpath = '/html/body/form/center/table/tbody/tr[3]/td[2]/input[1]'
    hit_save = driver.find_element(By.XPATH, xpath)
    hit_save.click()

    print(f"Cleared Sub for Part Number {sub_df['Part Number:'][i]} with Sub Value {sub_df['Part'][i]} at index {i}")
    
    # Coming back to Profile Tab to search for next part
    xpath = '/html/body/form/center/table/tbody/tr[1]/td/table/tbody/tr[2]/td[3]/table/tbody/tr/td[1]/a'
    profile_tab = driver.find_element(By.XPATH, xpath)
    profile_tab.click()

    time.sleep(2)

In [78]:
# We'll loop through each branch and create a sub dataframe for it
for branch in list(set(df.Br)):
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(3)

    print(f'Started Working on Branch: {branch}')
    sub_df = df[df.Br == branch]
    sub_df.reset_index(drop=True, inplace=True)
    
    # Branch Menu select to see what branches are available
    xpath = '//*[@id="navigation-top-bar"]/div[9]/div[1]'
    branch_select = driver.find_element(By.XPATH, xpath)
    branch_select.click()
    time.sleep(4)

    
    
    # Selecting Branch from list of branches
    # Branch 15 will be given value manually of 11 in the url rest other branches are consistent with branch number values and url
    if branch != 15:
        xpath = f'//*[@id="navigation-top-bar"]/div[9]/div[1]/div/div/navigation-menu-filter/div/navigation-menu-filter/div[1]/navigation-menu-filter/div[{branch}]/span/span'
        branch_select = driver.find_element(By.XPATH, xpath)
        branch_select.click()
        time.sleep(5)

    else:
        xpath = '//*[@id="navigation-top-bar"]/div[9]/div[1]/div/div/navigation-menu-filter/div/navigation-menu-filter/div[1]/navigation-menu-filter/div[11]/span/span'
        branch_select = driver.find_element(By.XPATH, xpath)
        branch_select.click()
        time.sleep(5)

    # Entering first part number just to open Update Parts Profile. We'l stay inside this dialogue box until next branch to speed up the search
    # Enter Part Number
    driver.switch_to.window(driver.window_handles[0])
    xpath = '/html/body/ui-view/div/div[3]/ui-view/div/listing-page/page-container/ng-transclude[1]/header/ng-transclude/filter/form/div/div/div[1]/cdk-form-field[1]/div/span/div/input'
    part_number = driver.find_element(By.XPATH, xpath)
    part_number.clear()
    part_number.send_keys(sub_df['Part Number:'][0])

    # Searching for Part and Description
    xpath = '//*[@id="navigation-placeholder"]/ui-view/div/listing-page/page-container/ng-transclude[1]/header/ng-transclude/filter/form/div/div/div[5]/div/button[2]'
    hit_search = driver.find_element(By.XPATH, xpath)
    hit_search.click()

    print(f'Switching to Update Profile Window for Branch {sub_df.Br[0]}')
    time.sleep(2)
    driver.switch_to.window(driver.window_handles[1])
    

     # Hit update profile option
    time.sleep(2)
    xpath = '/html/body/form/center/table[1]/tbody/tr/td/table[2]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr[2]/td[1]/a'
    update_profile_button = driver.find_element(By.XPATH, xpath)
    update_profile_button.click()

    print('Entered into Update Profile Dialogue Box')
    
    # Now for each branch we will run loops which will go line by line for all parts in the sub_df of that branch
    for i in sub_df.index:
        DeleteSubs()

Started Working on Branch: 1
Switching to Update Profile Window for Branch 1
Entered into Update Profile Dialogue Box
Cleared Sub for Part Number 503192984 with Sub Value 503192034 at index 0
Cleared Sub for Part Number 503192986 with Sub Value 503192182 at index 1
Cleared Sub for Part Number 503192988 with Sub Value 503191591 at index 2
Cleared Sub for Part Number 503192988 with Sub Value 503191887 at index 3
Cleared Sub for Part Number 503192991 with Sub Value 503191593 at index 4
Cleared Sub for Part Number 503193037 with Sub Value 503191422 at index 5
Cleared Sub for Part Number 503193098 with Sub Value 503191877 at index 6
Cleared Sub for Part Number 503193158 with Sub Value 503192136 at index 7
Cleared Sub for Part Number 503193159 with Sub Value 503192137 at index 8
Cleared Sub for Part Number 503193172 with Sub Value 503192770 at index 9
Cleared Sub for Part Number 503193173 with Sub Value 503192781 at index 10
